# Gimbal Lookup Table Calculator

This notebook gives tools to create a lookup table for each gimbal position. This lookup table is 2 dimensional where each axis describes a component of the command angle. The value associated with this position on the lookup table is a pair of servo commands which orient the gimbal into the command position.

This algorithm will first traverse all the possible servo angles and then output a table which represents the servo commands to gimbal position.

Currently this script is in it's primitive phase and only computes a gimbal angle for given servo commands.

© 2025 Ronan Howard. All rights reserved

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

In [2]:
# positions
# S_ca = np.array([14.5282132153,-23.6498906926,11.654])
# S_cb = np.array([-23.6498906926,14.5282132153,11.654])
S_ca = np.array([14.5282132153,-23.6498906926,8.654])
S_cb = np.array([-23.6498906926,14.5282132153,8.654])
r_S = 4.45
r_arm = 22.8336

# reference gimbal position
G_A_ref = np.array([22.555, 0, 15.8])
G_B_ref = np.array([0, 22.555, 15.8])
# theta_0 = 69.91886
theta_0 = 57.5421


In [ ]:
def find_orientation(t_A, t_B):
    theta_A = np.radians(t_A + theta_0)
    theta_B = np.radians(t_B + theta_0)

    S_A = S_ca + r_S * np.array([np.sin(theta_A), np.cos(theta_A), 0])
    S_B = S_cb + r_S * np.array([np.cos(theta_B), np.sin(theta_B), 0])

    # opptimization function
    def error(q):
        R = Rotation.from_quat(q).as_matrix()
        G_A = R @ G_A_ref
        G_B = R @ G_B_ref
        return (np.linalg.norm(G_A-S_A)-r_arm)**2 + (np.linalg.norm(G_B-S_B)-r_arm)**2

    # quaternion constraint
    def constraint(q):
        return q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2 - 1  # ||q|| = 1

    cons = {'type': 'eq', 'fun': constraint}

    # make initial guess (always starts with flush position)
    q0 = np.array([0,0,0,1]) # identity quaternion
    result = minimize(error, q0, method='SLSQP', constraints=cons)
    q_opt = result.x
    return Rotation.from_quat(q_opt).as_matrix()


def visualize_orientation(R):
    v = R @ [0,0,1]
    print(v)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(0,0,0, v[0], v[1], v[2])
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([0, 1])
    plt.show()

def visualize_orientation_vector(v):
    print(v)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(0,0,0, v[0], v[1], v[2])
    ax.set_xlim([-0.5, 0.5])
    ax.set_ylim([-0.5, 0.5])
    ax.set_zlim([0, 1])
    plt.show()

# Lookup table calcuation

In [7]:
servo_range = 10

servo_values = [x for x in range(-servo_range, servo_range + 1, 2)]

# this will be a 2 dimensional array
# first dimension describes the position of the servo A and the second describes the position of servo B
# the servo values are indexed from minimum to maximum
orientations = []
inverse_table = {}

for i in range(len(servo_values)):

    orientations.append([])
    
    for j in range(len(servo_values)):
        # find orientation and add to array
        orientations[i].append(find_orientation(servo_values[i], servo_values[j]) @ [0,0,1])

        # find associated gimbal command
        r = math.sqrt(orientations[i][j][0]**2 + orientations[i][j][1]**2)
        theta = math.atan2(orientations[i][j][1],orientations[i][j][0]) * (180/math.pi)

        inverse_table[(servo_values[i],servo_values[j])] = (r, theta)

lookup_table = {}

for servo_commands in inverse_table:
    lookup_table[inverse_table[servo_commands]] = servo_commands

# print(lookup_table)

In [8]:
print(lookup_table)

{(0.062302617156996684, 45.13510086804654): (-10, -10), (0.05684871998812986, 47.87232057630603): (-10, -8), (0.05152358477754953, 53.57342867841605): (-10, -6), (0.0448340458986597, 45.0721679109976): (-10, -4), (0.03876324566192119, 53.395618269322185): (-10, -2), (0.03221479441283435, 55.93062009172416): (-10, 0), (0.02593400888451659, 63.331378022897): (-10, 2), (0.02041682156404925, 77.09154562435411): (-10, 4), (0.012861892472679746, 85.68242310455197): (-10, 6), (0.010397998315655571, 124.22983608877844): (-10, 8), (0.011341670784125974, 167.49773054354375): (-10, 10), (0.05684862649776546, 42.129736022520056): (-8, -10), (0.05128006272555489, 45.23012228738112): (-8, -8), (0.046161314969819324, 55.92077321213247): (-8, -6), (0.046354248049471226, 79.24219222459988): (-8, -4), (0.03300410609396487, 52.236838974446385): (-8, -2), (0.026279537536929763, 53.81909281394727): (-8, 0), (0.018978595437217506, 51.095627579688056): (-8, 2), (0.01403964061981816, 79.70585333951105): (-8, 

# Find servo commands from lookup table

In [ ]:
r = 0
theta = 0

# super inefficient approach
# in the C++ program, a spacial indexing data structure should be used (most likely R-Tree or quadtree)

command_list = [key for key in lookup_table]
